<a href="https://colab.research.google.com/github/maxy808/CLIP_video/blob/frame-selection-xiaoyang/CLIP_video_0.1_frame8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -b frame-selection-xiaoyang https://github.com/maxy808/CLIP_video.git
%cd CLIP_video

Cloning into 'CLIP_video'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 112 (delta 54), reused 49 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 1.39 MiB | 3.80 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/CLIP_video


In [ ]:
# From CLIP
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install opencv-python boto3 requests pandas

/bin/bash: line 1: conda: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
# Connet to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Comment if the file exits
# !wget -O /content/drive/MyDrive/Data/msrvtt_data.zip https://github.com/ArrowLuo/CLIP4Clip/releases/download/v0.0/msrvtt_data.zip
# !unzip /content/drive/MyDrive/Data/msrvtt_data.zip -d /content/drive/MyDrive/Data/

In [ ]:
DATA_PATH = "/content/drive/MyDrive/Data/msrvtt_data"  # Directory of MSRVTT data
VIDEO_PATH = "/content/drive/MyDrive/Data/video"    # Directory of MSRVTT raw video

In [ ]:
import pandas as pd

# Load training dataset
train_path = f"{DATA_PATH}/MSRVTT_train.9k.csv"
df = pd.read_csv(train_path)

print("Size: ", len(df))
print("Column names", df.columns.tolist())
display(df.head())

# Choose a subset
def sample_subset(df, frac=0.1, random_state=42):
    return df.sample(frac=frac, random_state=random_state).reset_index(drop=True)

# Set fraction to 0.01
subset_df = sample_subset(df, frac=0.1)
print("Size of subset：", len(subset_df))
display(subset_df.head())


Size:  9000
Column names ['video_id']


,video_id
0,video0
1,video1
2,video2
3,video3
4,video4


Size of subset： 900


,video_id
0,video8405
1,video1162
2,video582
3,video4081
4,video9139


In [ ]:
# Save the subset
subset_path = f"{DATA_PATH}/MSRVTT_train.subset.csv"
subset_df.to_csv(subset_path, index=False)

print("Subset CSV saved to:", subset_path)

Subset CSV saved to: /content/drive/MyDrive/Data/msrvtt_data/MSRVTT_train.subset.csv


In [ ]:
import torch
print("Current device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU only")

Current device: NVIDIA A100-SXM4-40GB


In [ ]:
!wget -P ./modules https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt

--2025-04-24 19:27:01--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.253.70, 2620:1ec:bdf::70
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.253.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘./modules/ViT-B-32.pt.1’

ViT-B-32.pt.1       100%[===================>] 337.58M  87.3MB/s    in 3.8s    

2025-04-24 19:27:05 (89.6 MB/s) - ‘./modules/ViT-B-32.pt.1’ saved [353976522/353976522]



In [ ]:
torch.cuda.empty_cache()

In [ ]:
# Updata changes to files. Comment if the files are not changed
!cd CLIP_video && git pull

/bin/bash: line 1: cd: CLIP_video: No such file or directory


In [ ]:
!pip install ffmpeg
!python /content/CLIP_video/preprocess/compress_video.py --input_root "/content/drive/MyDrive/IDL/IDL Project/Data/video" --output_root "/content/drive/MyDrive/IDL/IDL Project/Data/compressed_video"

Total video need to process: 0
Begin with 12-core logical processor.
Compress finished, wait for checking files...


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

!python /content/CLIP_video/main_task_retrieval.py \
  --do_train \
  --num_thread_reader 8 \
  --epochs 5 \
  --batch_size 64 \
  --train_csv  "$DATA_PATH"/MSRVTT_train.subset.csv \
  --val_csv    "$DATA_PATH"/MSRVTT_JSFUSION_test.csv \
  --data_path  "$DATA_PATH"/MSRVTT_data.json \
  --features_path "$VIDEO_PATH" \
  --output_dir /content/CLIP_video/ckpts/ckpt_msrvtt_retrieval_looseType \
  --lr 1e-4 \
  --max_words 32 \
  --max_frames 8 \
  --batch_size_val 8 \
  --datatype msrvtt \
  --expand_msrvtt_sentences \
  --feature_framerate 1 \
  --coef_lr 1e-3 \
  --freeze_layer_num 6 \
  --slice_framepos 8 \
  --loose_type \
  --linear_patch 2d \
  --sim_header meanP \
  --pretrained_clip_name ViT-B/32 \
  --fp16 \
  --fp16_opt_level O1


04/24/2025 19:27:25 - INFO -   Effective parameters:
04/24/2025 19:27:25 - INFO -     <<< batch_size: 64
04/24/2025 19:27:25 - INFO -     <<< batch_size_val: 8
04/24/2025 19:27:25 - INFO -     <<< cache_dir: 
04/24/2025 19:27:25 - INFO -     <<< coef_lr: 0.001
04/24/2025 19:27:25 - INFO -     <<< cross_model: cross-base
04/24/2025 19:27:25 - INFO -     <<< cross_num_hidden_layers: 4
04/24/2025 19:27:25 - INFO -     <<< data_path: /content/drive/MyDrive/Data/msrvtt_data/MSRVTT_data.json
04/24/2025 19:27:25 - INFO -     <<< datatype: msrvtt
04/24/2025 19:27:25 - INFO -     <<< do_eval: False
04/24/2025 19:27:25 - INFO -     <<< do_lower_case: False
04/24/2025 19:27:25 - INFO -     <<< do_pretrain: False
04/24/2025 19:27:25 - INFO -     <<< do_train: True
04/24/2025 19:27:25 - INFO -     <<< epochs: 5
04/24/2025 19:27:25 - INFO -     <<< eval_frame_order: 0
04/24/2025 19:27:25 - INFO -     <<< expand_msrvtt_sentences: True
04/24/2025 19:27:25 - INFO -     <<< feature_framerate: 1
04/24/20